In [117]:
import numpy as np
import pandas as pd
import plotly.express as px

import calendar
from datetime import datetime

from sklearn.impute import KNNImputer

## 1.0 Global Data (Overview)

### 1.0.1 Global Quantities

In [261]:
df_global_q = pd.read_csv('Data/Global/global_exports_quantities.txt', delimiter='\t')
df_global_q

,Exporters,"2010-Exported quantity, Tons","2011-Exported quantity, Tons","2012-Exported quantity, Tons","2013-Exported quantity, Tons","2014-Exported quantity, Tons","2015-Exported quantity, Tons","2016-Exported quantity, Tons","2017-Exported quantity, Tons","2018-Exported quantity, Tons","2019-Exported quantity, Tons","2020-Exported quantity, Tons","2021-Exported quantity, Tons",Unnamed: 13
0,World,1214351.0,1153849.0,1214137,1267952.0,1326425,1370772,1435756,1386312,1466019,No Quantity,No Quantity,No Quantity,NaN
1,Netherlands,138693.0,141021.0,172551,175908.0,181423,191954,228026,222106,218488,237042,251917,272105,NaN
2,Ireland,130068.0,138898.0,140547,152629.0,141914,176444,185560,198444,218804,257186,289292,272013,NaN
3,New Zealand,245393.0,254906.0,265711,259878.0,296759,291884,297476,269147,273223,280773,242003,241038,NaN
4,Germany,110900.0,89595.0,99833,122346.0,129422,135736,129008,125805,116785,135311,133137,129049,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,Lesotho,0.0,0.0,0,0.0,0,0,0,0,0,0,0,NaN,NaN
171,Malawi,3.0,1.0,0,0.0,0,0,0,7,0,0,0,NaN,NaN
172,"Libya, State of",0.0,NaN,NaN,NaN,NaN,4,0,0,0,0,NaN,NaN,NaN
173,Mali,0.0,0.0,0,NaN,NaN,NaN,0,5,3,7,NaN,NaN,NaN


In [262]:
df_global_q.drop(df_global_q.filter(regex='^Unnamed').columns,axis=1,inplace=True)

# Remove the sum row 'World', the last row (blank) and the the last column (blank)
df_global_q = df_global_q[(df_global_q['Exporters'] != 'World')]

In [263]:
# Reshape the data, creating a row for each yearly observation
df_global_q = df_global_q.melt(id_vars=['Exporters'],
                               var_name='Year',
                               value_name='Quantity (tonnes)')

In [264]:
df_global_q

,Exporters,Year,Quantity (tonnes)
0,Netherlands,"2010-Exported quantity, Tons",138693.0
1,Ireland,"2010-Exported quantity, Tons",130068.0
2,New Zealand,"2010-Exported quantity, Tons",245393.0
3,Germany,"2010-Exported quantity, Tons",110900.0
4,Belgium,"2010-Exported quantity, Tons",74627.0
...,...,...,...
2083,Lesotho,"2021-Exported quantity, Tons",NaN
2084,Malawi,"2021-Exported quantity, Tons",NaN
2085,"Libya, State of","2021-Exported quantity, Tons",NaN
2086,Mali,"2021-Exported quantity, Tons",NaN


In [265]:
df_global_q['Year'] = df_global_q['Year'].str[:4]

In [266]:
df_global_q

,Exporters,Year,Quantity (tonnes)
0,Netherlands,2010,138693.0
1,Ireland,2010,130068.0
2,New Zealand,2010,245393.0
3,Germany,2010,110900.0
4,Belgium,2010,74627.0
...,...,...,...
2083,Lesotho,2021,NaN
2084,Malawi,2021,NaN
2085,"Libya, State of",2021,NaN
2086,Mali,2021,NaN


### 1.0.2 Global Values

In [267]:
df_global_v = pd.read_csv('Data/Global/global_exports_values.txt', delimiter='\t')
df_global_v

,Exporters,Exported value in 2010,Exported value in 2011,Exported value in 2012,Exported value in 2013,Exported value in 2014,Exported value in 2015,Exported value in 2016,Exported value in 2017,Exported value in 2018,Exported value in 2019,Exported value in 2020,Exported value in 2021,Unnamed: 13
0,World,3802865.0,4405425.0,3740252.0,4428320.0,4548405.0,4285212.0,4518147.0,6418706.0,6946564.0,6660120.0,5599323.0,6463730.0,NaN
1,Netherlands,445281.0,550349.0,529077.0,650138.0,636885.0,604647.0,696832.0,998508.0,1034068.0,980831.0,893428.0,1061956.0,NaN
2,Ireland,405769.0,518267.0,422993.0,541223.0,488388.0,543733.0,544280.0,891261.0,1089962.0,1086854.0,969339.0,1051065.0,NaN
3,New Zealand,688911.0,1026562.0,715697.0,756614.0,888665.0,797983.0,833396.0,1197924.0,1212271.0,1132339.0,881204.0,988288.0,NaN
4,Germany,357912.0,353460.0,326588.0,450429.0,456993.0,429577.0,407033.0,595252.0,571646.0,560655.0,477357.0,532870.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,Netherlands Antilles,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,"Libya, State of",0.0,NaN,NaN,NaN,NaN,31.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
172,Mali,0.0,0.0,3.0,NaN,NaN,NaN,0.0,11.0,40.0,44.0,NaN,NaN,NaN
173,Malawi,53.0,2.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,NaN,NaN


In [268]:
df_global_v.drop(df_global_v.filter(regex='^Unnamed').columns,axis=1,inplace=True)

# Remove the sum row 'World', the last row (blank) and the the last column (blank)
df_global_v = df_global_v[(df_global_v['Exporters'] != 'World')]

In [269]:
# Reshape the data, creating a row for each yearly observation
df_global_v = df_global_v.melt(id_vars=['Exporters'],
                               var_name='Year',
                               value_name='Value (k Euro)')

In [270]:
df_global_v['Year'] = df_global_v['Year'].str[-4:]

In [271]:
#df_global.sort_values(by=['Month Date','Value (k Euro)'],ascending=False,inplace=True)

### 1.0.3 Merge Global Quantities and Values

In [272]:
df_global = pd.merge(df_global_q,df_global_v,on=['Exporters','Year'], how='left')
df_global

,Exporters,Year,Quantity (tonnes),Value (k Euro)
0,Netherlands,2010,138693.0,445281.0
1,Ireland,2010,130068.0,405769.0
2,New Zealand,2010,245393.0,688911.0
3,Germany,2010,110900.0,357912.0
4,Belgium,2010,74627.0,261367.0
...,...,...,...,...
2083,Lesotho,2021,NaN,NaN
2084,Malawi,2021,NaN,NaN
2085,"Libya, State of",2021,NaN,NaN
2086,Mali,2021,NaN,NaN


In [273]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2088 entries, 0 to 2087
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Exporters          2088 non-null   object 
 1   Year               2088 non-null   object 
 2   Quantity (tonnes)  1872 non-null   object 
 3   Value (k Euro)     1872 non-null   float64
dtypes: float64(1), object(3)
memory usage: 81.6+ KB


In [274]:
# Remove rows where both quantity and value are null
df_global = df_global[(df_global['Quantity (tonnes)'].notna() & df_global['Value (k Euro)'].notna())]

In [275]:
# Check for nulls
df_global.isna().sum()

Exporters            0
Year                 0
Quantity (tonnes)    0
Value (k Euro)       0
dtype: int64

In [276]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1872 entries, 0 to 2057
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Exporters          1872 non-null   object 
 1   Year               1872 non-null   object 
 2   Quantity (tonnes)  1872 non-null   object 
 3   Value (k Euro)     1872 non-null   float64
dtypes: float64(1), object(3)
memory usage: 73.1+ KB


In [277]:
pd.options.mode.chained_assignment = None  # default='warn'
df_global['Quantity (tonnes)'] = df_global['Quantity (tonnes)'].apply(pd.to_numeric, errors='coerce')
df_global.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1872 entries, 0 to 2057
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Exporters          1872 non-null   object 
 1   Year               1872 non-null   object 
 2   Quantity (tonnes)  1858 non-null   float64
 3   Value (k Euro)     1872 non-null   float64
dtypes: float64(2), object(2)
memory usage: 73.1+ KB


In [278]:
# Check for nulls
df_global.isna().sum()

Exporters             0
Year                  0
Quantity (tonnes)    14
Value (k Euro)        0
dtype: int64

In [279]:
df_global[(df_global['Quantity (tonnes)'].isna())]

,Exporters,Year,Quantity (tonnes),Value (k Euro)
473,Viet Nam,2012,NaN,255.0
479,Bermuda,2012,NaN,1.0
794,Zimbabwe,2014,NaN,7.0
994,Nigeria,2015,NaN,6.0
1196,"Palestine, State of",2016,NaN,1.0
1370,"Palestine, State of",2017,NaN,10.0
1472,Namibia,2018,NaN,18.0
1576,United Kingdom,2019,NaN,183226.0
1658,Rwanda,2019,NaN,44.0
1876,Finland,2020,NaN,128984.0


In [280]:
# Use KNN Imputer to fill the missing values

column_names = ['Exporters','Year','Quantity (tonnes)','Value (k Euro)']

# Columns not included in KNN
fixed = np.array(df_global[column_names[:2]])
# Columns included in KNN
features = np.array(df_global[column_names[2:]])

impute_knn = KNNImputer(n_neighbors=1)
features = impute_knn.fit_transform(features)

full = np.append(fixed,features,axis=1)
df_global = pd.DataFrame(full,columns=['Exporters','Year','Quantity (tonnes)','Value (k Euro)'])

In [281]:
df_global

,Exporters,Year,Quantity (tonnes),Value (k Euro)
0,Netherlands,2010,138693.0,445281.0
1,Ireland,2010,130068.0,405769.0
2,New Zealand,2010,245393.0,688911.0
3,Germany,2010,110900.0,357912.0
4,Belgium,2010,74627.0,261367.0
...,...,...,...,...
1867,Suriname,2021,0.0,0.0
1868,Eswatini,2021,0.0,0.0
1869,Togo,2021,0.0,2.0
1870,United States Minor Outlying Islands,2021,0.0,1.0


### 1.0.4 Plot Global Data

In [282]:
# df_global.sort_values(by=['Year','Quantity (tonnes)'],ascending=[True,False],inplace=True)
fig = px.line(df_global, 
              x="Year", 
              y="Quantity (tonnes)", 
              color='Exporters', 
              title='Butter Export Quantities - All Countries')
fig.show()

In [283]:
# df_global.sort_values(by=['Year','Value (k Euro)'],ascending=False,inplace=True)
fig = px.line(df_global, 
              x='Year', 
              y='Value (k Euro)', 
              color='Exporters', 
              title='Butter Export Values - All Countries')
fig.show()

## 1.1 EU Data

In [20]:
df_eu = pd.read_csv('Data/EU_MILK_trade_data_en.csv')
df_eu.head()

,Flow,Marketing Year,Month,Month Order in MY,Month Date,Member State,Partner,Product Group,Product Code (CN),Quantity in tonnes,Value in thousand euro
0,EXPORT,2022,Oct,10,01/10/2022,Austria,Albania,Cheese,4063039,17.677,86.192
1,EXPORT,2022,Oct,10,01/10/2022,Austria,Angola,Cheese,4061050,0.472,2.189
2,EXPORT,2022,Oct,10,01/10/2022,Austria,Angola,Cheese,4063031,9.862,41.178
3,EXPORT,2022,Oct,10,01/10/2022,Austria,Angola,Cheese,4063039,3.177,14.467
4,EXPORT,2022,Oct,10,01/10/2022,Austria,Antigua and Barbuda,Cheese,4063031,1.800,9.212


In [21]:
df = df_eu[(df_eu['Member State']=='Ireland') & 
           (df_eu['Product Code (CN)'].astype('string').str[:5]=='40510') & 
           (df_eu['Flow']=='EXPORT')]
df

,Flow,Marketing Year,Month,Month Order in MY,Month Date,Member State,Partner,Product Group,Product Code (CN),Quantity in tonnes,Value in thousand euro
5501,EXPORT,2022,Oct,10,01/10/2022,Ireland,Australia,Butter,4051011,20.060,142.579
5503,EXPORT,2022,Oct,10,01/10/2022,Ireland,Bahrain,Butteroil,4051090,0.054,0.176
5521,EXPORT,2022,Oct,10,01/10/2022,Ireland,Cayman islands,Butter,4051011,9.355,86.842
5523,EXPORT,2022,Oct,10,01/10/2022,Ireland,China,Butter,4051011,23.280,141.778
5549,EXPORT,2022,Oct,10,01/10/2022,Ireland,Gibraltar,Butter,4051011,0.960,8.074
...,...,...,...,...,...,...,...,...,...,...,...
1463273,EXPORT,2010,Jan,1,01/01/2010,Ireland,Gibraltar,Butter,4051011,5.294,17.755
1463297,EXPORT,2010,Jan,1,01/01/2010,Ireland,Oman,Butter,4051011,9.759,33.768
1463302,EXPORT,2010,Jan,1,01/01/2010,Ireland,South Africa,Butter,4051011,40.936,147.687
1463311,EXPORT,2010,Jan,1,01/01/2010,Ireland,Trinidad and Tobago,Butter,4051011,17.335,62.713


In [22]:
df = df.groupby(['Flow','Partner','Month Date','Month','Marketing Year']).sum().reset_index()
df

,Flow,Partner,Month Date,Month,Marketing Year,Month Order in MY,Product Code (CN),Quantity in tonnes,Value in thousand euro
0,EXPORT,Algeria,01/01/2011,Jan,2011,1,4051019,208.000,707.200
1,EXPORT,Algeria,01/01/2015,Jan,2015,1,4051019,50.000,151.500
2,EXPORT,Algeria,01/01/2019,Jan,2019,1,4051019,100.000,415.000
3,EXPORT,Algeria,01/01/2020,Jan,2020,1,4051019,49.750,178.603
4,EXPORT,Algeria,01/02/2017,Feb,2017,2,4051019,25.000,104.500
...,...,...,...,...,...,...,...,...,...
2603,EXPORT,Zambia,01/07/2012,Jul,2012,7,4051011,7.200,36.310
2604,EXPORT,Zambia,01/12/2011,Dec,2011,12,4051011,9.169,43.118
2605,EXPORT,Zimbabwe,01/04/2016,Apr,2016,4,4051011,5.000,18.513
2606,EXPORT,Zimbabwe,01/09/2013,Sep,2013,9,4051011,8.490,47.467


In [23]:
df = df.groupby(['Marketing Year']).sum().reset_index()

In [24]:
df

,Marketing Year,Month Order in MY,Product Code (CN),Quantity in tonnes,Value in thousand euro
0,2010,996,603601152,7956.961,28006.582
1,2011,740,510427720,5687.297,24287.729
2,2012,949,571192893,6908.598,26919.962
3,2013,946,579295037,5595.613,24317.852
4,2014,1251,753489276,9427.842,38586.376
5,2015,1704,1016806060,22176.664,79432.879
6,2016,1831,1178846307,33485.652,110904.084
7,2017,1740,1118080728,31536.196,144567.345
8,2018,1301,895274164,31161.031,195874.678
9,2019,2285,1324682720,50952.124,283954.787


In [25]:
df[(df['Year'] = pd.DatetimeIndex(df_nz['Month Date']).year

SyntaxError: invalid syntax (1580646419.py, line 1)

In [324]:
df_eu = df_eu[(df_eu['Product Code (CN)'].astype('string').str[:5]=='40510') & (df_eu['Flow']=='EXPORT')]

In [325]:
df_eu.isna().sum()

Flow                      0
Marketing Year            0
Month                     0
Month Order in MY         0
Month Date                0
Member State              0
Partner                   0
Product Group             0
Product Code (CN)         0
Quantity in tonnes        0
Value in thousand euro    0
dtype: int64

In [326]:
df_eu['Product Group'].unique()

array(['Butter', 'Butteroil'], dtype=object)

In [327]:
# Use lambda function to convert the Month Date values to datetime
df_eu['Month Date'] = df_eu['Month Date'].map(lambda x: datetime.strptime(x, '%d/%m/%Y'))


In [328]:
df_eu[(df_eu['Member State']=='Austria')]

,Flow,Marketing Year,Month,Month Order in MY,Month Date,Member State,Partner,Product Group,Product Code (CN),Quantity in tonnes,Value in thousand euro
35,EXPORT,2022,Oct,10,2022-10-01,Austria,Holy See (Vatican City State),Butter,4051011,1.800,14.275
89,EXPORT,2022,Oct,10,2022-10-01,Austria,Switzerland,Butter,4051011,0.010,0.084
90,EXPORT,2022,Oct,10,2022-10-01,Austria,Switzerland,Butteroil,4051090,0.000,0.007
130,EXPORT,2022,Oct,10,2022-10-01,Austria,United Arab Emirates,Butter,4051011,0.376,1.564
10165,EXPORT,2022,Sep,9,2022-09-01,Austria,Holy See (Vatican City State),Butter,4051011,1.200,9.535
...,...,...,...,...,...,...,...,...,...,...,...
1459527,EXPORT,2010,Jan,1,2010-01-01,Austria,Russia,Butter,4051011,1.272,6.639
1459553,EXPORT,2010,Jan,1,2010-01-01,Austria,Switzerland,Butter,4051019,0.479,1.242
1459554,EXPORT,2010,Jan,1,2010-01-01,Austria,Switzerland,Butteroil,4051090,0.128,0.465
1459605,EXPORT,2010,Jan,1,2010-01-01,Austria,Turkey,Butteroil,4051090,0.009,0.050


In [329]:
df_eu.rename(columns = {'Marketing Year':'Year', 
                        'Member State':'Country',
                        'Product Code (CN)':'Product',
                        'Quantity in tonnes':'Quantity (tonnes)',
                        'Value in thousand euro':'Value (k Euro)'
                       }, inplace = True)

In [330]:
df_eu.drop(['Month Order in MY','Product Group'], axis=1,inplace=True)

## 1.1 Ireland Data

In [ ]:
ie_files = [
    {
        'filename':'Data/IE/ie_exports_quantities_01.txt',
        'filenames':['Data/IE/ie_exports_quantities_01.txt',
                    'Data/IE/ie_exports_quantities_02.txt',
                    'Data/IE/ie_exports_quantities_03.txt',
                    'Data/IE/ie_exports_quantities_04.txt',
                    'Data/IE/ie_exports_quantities_05.txt',
                    'Data/IE/ie_exports_quantities_06.txt',
                    'Data/IE/ie_exports_quantities_07.txt',
                    'Data/IE/ie_exports_quantities_08.txt',
                    'Data/IE/ie_exports_quantities_09.txt'],
        'partner_column':'Importers',
        'column_name':'Quantity in tonnes',
        'flow':'EXPORT',
        'left_date_text':'',
        'right_date_text':'%Y-M%m-Exported quantity, Kilograms',
        'date_format':'%Y-M%m'
    },
    {
        'filename':'Data/IE/ie_exports_values_01.txt',
        'filenames':['Data/IE/ie_exports_values_01.txt',
                    'Data/IE/ie_exports_values_02.txt',
                    'Data/IE/ie_exports_values_03.txt',
                    'Data/IE/ie_exports_values_04.txt',
                    'Data/IE/ie_exports_values_05.txt',
                    'Data/IE/ie_exports_values_06.txt',
                    'Data/IE/ie_exports_values_07.txt',
                    'Data/IE/ie_exports_values_08.txt',
                    'Data/IE/ie_exports_values_09.txt'],
        'partner_column':'Importers',
        'column_name':'Value in thousand euro',
        'flow':'EXPORT',
        'left_date_text':'Exported value in ',
        'right_date_text':'',
        'date_format':'%Y-M%m'
    }
]

## 1.2 NewZealand Data

In [331]:
nz_files = [
    {
        'filename':'Data/NZ/nz_exports_quantities_01.txt',
        'filenames':['Data/NZ/nz_exports_quantities_01.txt',
                    'Data/NZ/nz_exports_quantities_02.txt',
                    'Data/NZ/nz_exports_quantities_03.txt',
                    'Data/NZ/nz_exports_quantities_04.txt',
                    'Data/NZ/nz_exports_quantities_05.txt',
                    'Data/NZ/nz_exports_quantities_06.txt',
                    'Data/NZ/nz_exports_quantities_07.txt',
                    'Data/NZ/nz_exports_quantities_08.txt'],
        'partner_column':'Importers',
        'column_name':'Quantity in tonnes',
        'flow':'EXPORT',
        'left_date_text':'',
        'right_date_text':'%Y-M%m-Exported quantity, Kilograms',
        'date_format':'%Y-M%m'
    },
    {
        'filename':'Data/NZ/nz_exports_values_01.txt',
        'filenames':['Data/NZ/nz_exports_values_01.txt',
                    'Data/NZ/nz_exports_values_02.txt',
                    'Data/NZ/nz_exports_values_03.txt',
                    'Data/NZ/nz_exports_values_04.txt',
                    'Data/NZ/nz_exports_values_05.txt',
                    'Data/NZ/nz_exports_values_06.txt',
                    'Data/NZ/nz_exports_values_07.txt',
                    'Data/NZ/nz_exports_values_08.txt'],
        'partner_column':'Importers',
        'column_name':'Value in thousand euro',
        'flow':'EXPORT',
        'left_date_text':'Exported value in ',
        'right_date_text':'',
        'date_format':'%Y-M%m'
    },
    {
        'filename':'Data/NZ/nz_imports_quantities_01.txt',
        'filenames':['Data/NZ/nz_imports_quantities_01.txt',
                    'Data/NZ/nz_imports_quantities_02.txt',
                    'Data/NZ/nz_imports_quantities_03.txt',
                    'Data/NZ/nz_imports_quantities_04.txt',
                    'Data/NZ/nz_imports_quantities_05.txt',
                    'Data/NZ/nz_imports_quantities_06.txt',
                    'Data/NZ/nz_imports_quantities_07.txt',
                    'Data/NZ/nz_imports_quantities_08.txt'],
        'partner_column':'Exporters',
        'column_name':'Quantity in tonnes',
        'flow':'IMPORT',
        'left_date_text':'',
        'right_date_text':'-Imported quantity, Kilograms',
        'date_format':'%Y-M%m'
    },
    {
        'filename':'Data/NZ/nz_imports_values_01.txt',
        'filenames':['Data/NZ/nz_imports_values_01.txt',
                    'Data/NZ/nz_imports_values_02.txt',
                    'Data/NZ/nz_imports_values_03.txt',
                    'Data/NZ/nz_imports_values_04.txt',
                    'Data/NZ/nz_imports_values_05.txt',
                    'Data/NZ/nz_imports_values_06.txt',
                    'Data/NZ/nz_imports_values_07.txt',
                    'Data/NZ/nz_imports_values_08.txt'],
        'partner_column':'Exporters',
        'column_name':'Value in thousand euro',
        'flow':'IMPORT',
        'left_date_text':'Imported value in ',
        'right_date_text':'',
        'date_format':'%Y-M%m'
    }
]

In [332]:
first_quantity_data = True
first_value_data = True

partner_column_name = 'Partner'

for index in range(len(nz_files)):
    #df = pd.read_csv(nz_files[index]['filename'], sep='\t', lineterminator='\r')
    first_part = True
    for i in range(len(nz_files[index]['filenames'])):
        df_part = pd.read_csv(nz_files[index]['filenames'][i], sep='\t', lineterminator='\r')
        key = nz_files[index]['partner_column']
        
        # Rename the 'Importers' / 'Exporters' column to 'Partner', in line with the EU data 
        df_part.rename(columns = {key:'Partner'}, inplace = True)
        #df_part.columns.values[key] = 'Partner'
        
        if first_part:
            df_parts = df_part
            first_part = False
        else:
            df_parts = df_parts.merge(df_part, on='Partner',how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
            df_parts.drop(df_parts.filter(regex='^Unnamed').columns,axis=1,inplace=True)
            df_parts = df_parts[(df_parts['Partner'] != '\n')]
                   
    df = df_parts
    
    # Use lambda function to strip unwanted characters from beginning (lstrip) 
    # and end (rstrip) of the Importers values
    df['Partner'] = df['Partner'].map(lambda x: x.lstrip('\n"').rstrip('"'))
    
    # Remove the sum row 'World', the last row (blank) and the the last column (blank)
    df = df[(df['Partner'] != 'World')]
    
    # Reshape the data, creating a row for each monthly observation
    df = df.melt(id_vars=['Partner'], var_name='Month Date', value_name=nz_files[index]['column_name'])
    
    # Add a 'Flow' column to indicate IMPORT or EXPORT
    df['Flow'] = nz_files[index]['flow']
    
    
    # Use lambda function to strip unwanted text and convert Month Date values to datetime
    df['Month Date'] = df['Month Date'].map(lambda x: x.lstrip(nz_files[index]['left_date_text']).rstrip(nz_files[index]['right_date_text']))
    df['Month Date'] = df['Month Date'].map(lambda x: datetime.strptime(x, nz_files[index]['date_format']))
    
    if nz_files[index]['column_name'] == 'Value in thousand euro':
        if first_value_data:
            df_values = df
            first_value_data = False
        else:
            df_values = pd.concat([df_values,df],ignore_index=True)
    else:
        if first_quantity_data:
            df_quantities = df
            first_quantity_data = False
        else:
            df_quantities = pd.concat([df_quantities,df],ignore_index=True)
    
    
df_nz = pd.merge(df_values,
                 df_quantities[['Month Date','Partner','Flow','Quantity in tonnes']],
                 on=['Month Date','Partner','Flow'], 
                 how='left')
df_nz

,Partner,Month Date,Value in thousand euro,Flow,Quantity in tonnes
0,Denmark,2010-01-01,8271.0,EXPORT,4332245.0
1,Russian Federation,2010-01-01,5504.0,EXPORT,2008407.0
2,Morocco,2010-01-01,4296.0,EXPORT,2334257.0
3,China,2010-01-01,3211.0,EXPORT,1578730.0
4,Australia,2010-01-01,4608.0,EXPORT,2219114.0
...,...,...,...,...,...
16978,Fiji,2022-09-01,0.0,IMPORT,0.0
16979,France,2022-09-01,0.0,IMPORT,0.0
16980,New Zealand,2022-09-01,544.0,IMPORT,98542.0
16981,India,2022-09-01,4.0,IMPORT,750.0


In [333]:
df_nz['Year'] = pd.DatetimeIndex(df_nz['Month Date']).year
df_nz['Month'] = pd.DatetimeIndex(df_nz['Month Date']).month
df_nz['Month'] = df_nz['Month'].apply(lambda x: calendar.month_abbr[x])

In [334]:
df_nz.rename(columns = {
                        'Quantity in tonnes':'Quantity (tonnes)',
                        'Value in thousand euro':'Value (k Euro)'
                       }, inplace = True)
df_nz['Quantity (tonnes)'] = df_nz['Quantity (tonnes)'] / 1000

In [335]:
df_nz['Country'] = 'New Zealand'

In [336]:
df_nz

,Partner,Month Date,Value (k Euro),Flow,Quantity (tonnes),Year,Month,Country
0,Denmark,2010-01-01,8271.0,EXPORT,4332.245,2010,Jan,New Zealand
1,Russian Federation,2010-01-01,5504.0,EXPORT,2008.407,2010,Jan,New Zealand
2,Morocco,2010-01-01,4296.0,EXPORT,2334.257,2010,Jan,New Zealand
3,China,2010-01-01,3211.0,EXPORT,1578.730,2010,Jan,New Zealand
4,Australia,2010-01-01,4608.0,EXPORT,2219.114,2010,Jan,New Zealand
...,...,...,...,...,...,...,...,...
16978,Fiji,2022-09-01,0.0,IMPORT,0.000,2022,Sep,New Zealand
16979,France,2022-09-01,0.0,IMPORT,0.000,2022,Sep,New Zealand
16980,New Zealand,2022-09-01,544.0,IMPORT,98.542,2022,Sep,New Zealand
16981,India,2022-09-01,4.0,IMPORT,0.750,2022,Sep,New Zealand


## 1.3 Combine Data


In [337]:
df_eu.drop(columns='Product',axis=1,inplace=True)

df_eu = df_eu.groupby(['Country','Flow','Partner','Month Date','Month','Year']).sum().reset_index()

In [338]:
df_eu.head(3)

,Country,Flow,Partner,Month Date,Month,Year,Quantity (tonnes),Value (k Euro)
0,Austria,EXPORT,Azerbaijan,2016-08-01,Aug,2016,7.376,27.301
1,Austria,EXPORT,Bahrain,2011-01-01,Jan,2011,0.043,0.100
2,Austria,EXPORT,Bosnia and Herzegovina,2021-09-01,Sep,2021,0.001,0.003


In [339]:
df_nz = df_nz[['Country','Flow','Partner','Month Date','Month','Year','Quantity (tonnes)','Value (k Euro)']]

In [340]:
df_nz.head(3)

,Country,Flow,Partner,Month Date,Month,Year,Quantity (tonnes),Value (k Euro)
0,New Zealand,EXPORT,Denmark,2010-01-01,Jan,2010,4332.245,8271.0
1,New Zealand,EXPORT,Russian Federation,2010-01-01,Jan,2010,2008.407,5504.0
2,New Zealand,EXPORT,Morocco,2010-01-01,Jan,2010,2334.257,4296.0


In [341]:
pd.set_option('display.max_rows', 20)           

In [342]:
df_eu_nz = pd.concat([df_eu,df_nz],ignore_index=True)

In [343]:
df_eu_nz

,Country,Flow,Partner,Month Date,Month,Year,Quantity (tonnes),Value (k Euro)
0,Austria,EXPORT,Azerbaijan,2016-08-01,Aug,2016,7.376,27.301
1,Austria,EXPORT,Bahrain,2011-01-01,Jan,2011,0.043,0.100
2,Austria,EXPORT,Bosnia and Herzegovina,2021-09-01,Sep,2021,0.001,0.003
3,Austria,EXPORT,Canada,2020-11-01,Nov,2020,0.000,0.017
4,Austria,EXPORT,China,2013-11-01,Nov,2013,2.000,13.900
...,...,...,...,...,...,...,...,...
69078,New Zealand,IMPORT,Fiji,2022-09-01,Sep,2022,0.000,0.000
69079,New Zealand,IMPORT,France,2022-09-01,Sep,2022,0.000,0.000
69080,New Zealand,IMPORT,New Zealand,2022-09-01,Sep,2022,98.542,544.000
69081,New Zealand,IMPORT,India,2022-09-01,Sep,2022,0.750,4.000


In [344]:
df_eu_nz[(df_eu_nz['Flow']=='IMPORT')]

,Country,Flow,Partner,Month Date,Month,Year,Quantity (tonnes),Value (k Euro)
68012,New Zealand,IMPORT,Australia,2010-01-01,Jan,2010,12.596,53.0
68013,New Zealand,IMPORT,Denmark,2010-01-01,Jan,2010,7.024,32.0
68014,New Zealand,IMPORT,Fiji,2010-01-01,Jan,2010,0.000,0.0
68015,New Zealand,IMPORT,France,2010-01-01,Jan,2010,0.046,0.0
68016,New Zealand,IMPORT,New Zealand,2010-01-01,Jan,2010,0.000,0.0
...,...,...,...,...,...,...,...,...
69078,New Zealand,IMPORT,Fiji,2022-09-01,Sep,2022,0.000,0.0
69079,New Zealand,IMPORT,France,2022-09-01,Sep,2022,0.000,0.0
69080,New Zealand,IMPORT,New Zealand,2022-09-01,Sep,2022,98.542,544.0
69081,New Zealand,IMPORT,India,2022-09-01,Sep,2022,0.750,4.0


In [345]:
df_nz[(df_nz['Flow']=='IMPORT')]

,Country,Flow,Partner,Month Date,Month,Year,Quantity (tonnes),Value (k Euro)
15912,New Zealand,IMPORT,Australia,2010-01-01,Jan,2010,12.596,53.0
15913,New Zealand,IMPORT,Denmark,2010-01-01,Jan,2010,7.024,32.0
15914,New Zealand,IMPORT,Fiji,2010-01-01,Jan,2010,0.000,0.0
15915,New Zealand,IMPORT,France,2010-01-01,Jan,2010,0.046,0.0
15916,New Zealand,IMPORT,New Zealand,2010-01-01,Jan,2010,0.000,0.0
...,...,...,...,...,...,...,...,...
16978,New Zealand,IMPORT,Fiji,2022-09-01,Sep,2022,0.000,0.0
16979,New Zealand,IMPORT,France,2022-09-01,Sep,2022,0.000,0.0
16980,New Zealand,IMPORT,New Zealand,2022-09-01,Sep,2022,98.542,544.0
16981,New Zealand,IMPORT,India,2022-09-01,Sep,2022,0.750,4.0


## 2.1 Plot

In [346]:
df_plot = df_eu_nz.copy() # Use deep copy to preserve the contents of df_eu 

In [347]:
df_plot = df_plot[(df_plot['Flow']=='EXPORT')]

In [348]:
df_plot.drop(
    ['Flow','Year','Month','Partner'],
    axis=1,
    inplace=True)


In [349]:
df_plot

,Country,Month Date,Quantity (tonnes),Value (k Euro)
0,Austria,2016-08-01,7.376,27.301
1,Austria,2011-01-01,0.043,0.100
2,Austria,2021-09-01,0.001,0.003
3,Austria,2020-11-01,0.000,0.017
4,Austria,2013-11-01,2.000,13.900
...,...,...,...,...
68007,New Zealand,2022-09-01,7.586,61.000
68008,New Zealand,2022-09-01,NaN,NaN
68009,New Zealand,2022-09-01,NaN,NaN
68010,New Zealand,2022-09-01,0.000,0.000


In [350]:
df_plot = df_plot.groupby(['Country','Month Date']).sum().reset_index()

In [351]:
df_plot

,Country,Month Date,Quantity (tonnes),Value (k Euro)
0,Austria,2010-01-01,7.529,33.448
1,Austria,2010-02-01,12.382,56.602
2,Austria,2010-03-01,7.520,34.685
3,Austria,2010-04-01,6.950,28.078
4,Austria,2010-05-01,0.698,3.211
...,...,...,...,...
3734,United Kingdom,2019-09-01,730.051,2947.043
3735,United Kingdom,2019-10-01,1212.479,4995.317
3736,United Kingdom,2019-11-01,1252.873,5139.480
3737,United Kingdom,2019-12-01,620.075,2575.875


In [352]:
# df_nz

In [353]:
# df_nz['Member State'] = 'New Zealand'
# df_nz['Value in thousand euro'] = 0

In [354]:
# df_nz

### Concatenate both sets together

In [355]:
# df_plot = pd.concat([df_plot,df_nz],ignore_index=True)

In [356]:
df_plot = df_plot.groupby([pd.Grouper(key='Month Date',freq='1Y'),'Country']).sum().reset_index()

In [357]:
df_plot

,Month Date,Country,Quantity (tonnes),Value (k Euro)
0,2010-12-31,Austria,78.552,356.651
1,2010-12-31,Belgium,9822.300,34837.812
2,2010-12-31,Bulgaria,80.475,219.644
3,2010-12-31,Czechia,869.249,3054.372
4,2010-12-31,Denmark,9798.743,44552.686
...,...,...,...,...
345,2022-12-31,Romania,176.248,1182.574
346,2022-12-31,Slovakia,53.646,393.132
347,2022-12-31,Slovenia,89.055,585.854
348,2022-12-31,Spain,1094.802,8036.446


In [358]:
df_plot.sort_values(by=['Month Date','Value (k Euro)'],ascending=False,inplace=True)

In [359]:
df_plot

,Month Date,Country,Quantity (tonnes),Value (k Euro)
342,2022-12-31,New Zealand,178756.266,1014677.000
335,2022-12-31,Ireland,68871.577,499476.726
331,2022-12-31,France,40360.689,298713.263
328,2022-12-31,Denmark,24279.438,154833.218
341,2022-12-31,Netherlands,17732.504,121849.368
...,...,...,...,...
2,2010-12-31,Bulgaria,80.475,219.644
10,2010-12-31,Hungary,21.858,93.619
9,2010-12-31,Greece,0.929,9.473
15,2010-12-31,Luxembourg,0.840,2.536


In [362]:
fig = px.line(df_plot, x="Month Date", y="Quantity (tonnes)", color='Country', title='Butter Exports')
fig.show()

In [361]:
fig = px.line(df_plot, x="Month Date", y="Value (k Euro)", color='Country', title='Butter Exports')
fig.show()